In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import sys
from pandas import HDFStore,DataFrame

import matplotlib.pyplot as plt
#from matplotlib_venn import venn2


# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)

product_level = 'Product Level 2'


In [2]:
def get_data_frame_summmary(data_frame):
        unique_values = data_frame.apply(lambda x: [x.unique()])
        unique_counts = data_frame.apply(lambda x: len(x.unique()))
        percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
        data_type = data_frame.dtypes 

        return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by='percent_missing', ascending=False)

In [3]:
# hdf = HDFStore('../data/hdf/datastore.h5')

# print(hdf.keys())
# hdf.close()

In [3]:
# Define CRM and UPL contracts
crm_list = ['Active Renewal', 'Cancelled', 'Evegreen Renewal', 'Merged', 'New', 'Standard Renewal']
upl_pattern = 'UPL-5'

In [5]:
# Read Data
ECH = pd.read_hdf('../data/hdf/datastore.h5', 'ECH_Customer_Data')
hierarchy = pd.read_hdf('../data/hdf/datastore.h5', 'hierarchy')

journals_contracts = pd.read_hdf('../data/hdf/datastore.h5', 'DataCRjournals')
other_contracts = pd.read_hdf('../data/hdf/datastore.h5', 'DataCRother')

churn_activities = pd.read_hdf('../data/hdf/datastore.h5', 'churn_activities')
churn_risks = pd.read_hdf('../data/hdf/datastore.h5', 'churn_risks_V02')

account_assignment = pd.read_hdf('../data/hdf/datastore.h5', 'Account_Assignment')
NPS = pd.read_hdf('../data/hdf/datastore.h5', 'NPS_Cleansed_Data')

churn_products = pd.read_hdf('../data/hdf/datastore.h5', 'churn_products')
product_assignment = pd.read_hdf('../data/hdf/datastore.h5', 'Product_Assignment_2019')

usage = pd.read_hdf('../data/hdf/datastore.h5', 'usage')
interactions = pd.read_hdf('../data/hdf/datastore.h5', 'interaction')
cancellations = pd.read_hdf('../data/hdf/datastore.h5', 'DataCR_from_2015_Cancellations_Source_Systems')


sis_ecr_mapping = pd.read_hdf('../data/hdf/datastore.h5', 'sis_mapping')


 
### KEEP ONLY CRM AND UPL DATA

### Remove non-RSS transactions

In [6]:
is_crm_journals = journals_contracts['Sales Type'].isin(crm_list)
is_upl_journals = journals_contracts['Agreement Number'].str.startswith(upl_pattern).fillna(False)
print(is_crm_journals.value_counts())
print(is_upl_journals.value_counts())

True     2530925
False    1613197
Name: Sales Type, dtype: int64
False    4143836
True         286
Name: Agreement Number, dtype: int64


In [7]:
# Keep only crm or upl agreements
journals_contracts = journals_contracts[is_crm_journals | is_upl_journals]

journals_contracts['Sales Type'].value_counts(dropna=False)

Standard Renewal    1431211
Active Renewal      1015124
New                   52028
Merged                31654
Cancelled               908
nan                     286
Name: Sales Type, dtype: int64

In [4]:
is_crm_others = other_contracts['Sales Type'].isin(crm_list)
is_upl_others = other_contracts['Agreement Number'].str.startswith(upl_pattern).fillna(False)
print(is_crm_others.value_counts())
print(is_upl_others.value_counts())

False    10205541
True       697522
Name: Sales Type, dtype: int64
False    10893409
True         9654
Name: Agreement Number, dtype: int64


In [5]:
# Keep only crm or upl agreements
other_contracts = other_contracts[is_crm_others | is_upl_others]

other_contracts['Sales Type'].value_counts(dropna=False)

New                 387748
Active Renewal      173875
Standard Renewal    111507
Cancelled            15998
nan                   9654
Merged                8394
Name: Sales Type, dtype: int64

## Keep only RSS products

In [9]:
rss_products = churn_products[product_level].unique()
is_rss_product = journals_contracts['Product Line Level 2'].isin(rss_products)

In [11]:
is_rss_product.value_counts()

True    2531211
Name: Product Line Level 2, dtype: int64

In [12]:
journals_contracts = journals_contracts[is_rss_product]

In [10]:
is_rss_product = other_contracts['Product Line Level 2'].isin(rss_products)
is_rss_product.value_counts()

True     535270
False    171906
Name: Product Line Level 2, dtype: int64

In [11]:
other_contracts = other_contracts[is_rss_product]

## Check if ECR IDs

In [15]:
is_ecrid = journals_contracts['HQ SIS Id (Agreement SIS)'].str.startswith('ECR-').fillna(False)

In [16]:
is_ecrid.value_counts()

True     2531190
False         21
Name: HQ SIS Id (Agreement SIS), dtype: int64

### Pick Value from parent agreement

In [17]:
usage.head()

,ACT_CLICK_DEPTH,ACT_DWELL_TIME_VISIT_MIN,ECR_ID,LOY_DWELL_TIME_USER_MIN,LOY_RETURN_RATE,POP_ACTIVE_USERS,POP_PAGE_VIEWS,POP_TIME_SPENT_HRS,POP_VISITS,PROD_NAME,REPORT_AGG,REPORT_DT
0,0.0,0.0,ECR-1003,0.0,0.00,0,0,0.00,0,SCOPUS,MONTH,2016-09-01
1,0.0,0.0,ECR-1003,0.0,0.00,0,0,0.00,0,SCOPUS,MONTH,2017-05-01
2,0.0,0.003968253968,ECR-1003,0.004166666667,1.05,60,0,0.00,63,SCOPUS,MONTH,2018-11-01
3,4.731138546,4.558527663466,ECR-1003,6.354047163799,1.39,523,3449,55.39,729,SCOPUS,MONTH,2019-06-01
4,0.0,0.0,ECR-10036481,0.0,0.00,0,0,0.00,0,SCOPUS,MONTH,2016-08-01


In [18]:
is_ecrid_other = other_contracts['HQ SIS Id (Agreement SIS)'].str.startswith('ECR-').fillna(False)

In [19]:
is_ecrid_other.value_counts()

True     535255
False        15
Name: HQ SIS Id (Agreement SIS), dtype: int64

In [20]:
journals_contracts.columns

Index(['Agreement End Date', 'Agreement Number', 'Agreement Start Date',
       'Bookigns - Committed Print(Rep)',
       'Bookings - Final Net Price - Agent Discount Amount(Rep)',
       'Business Division (Agreement SIS)', 'Business Indicator',
       'Calculated New/Renewal', 'Country Name (Agreement SIS)', 'Division',
       'HQ SIS Id (Agreement SIS)', 'Invoice Date', 'Invoice Num',
       'Name  (Agreement SIS)', 'Parent Agreement Number', 'Payment Term',
       'Payment Term Description', 'Payment Term Type', 'Product Line Level 1',
       'Product Line Level 2', 'Product Line Level 3', 'Product Line Level 4',
       'Product Revenue Type', 'RSO', 'Renewal Exp Complete Date',
       'SIS Id  (Agreement SIS)', 'Saleable Product Name (Source)',
       'Sales Division (Agreement SIS)', 'Sales Type', 'Status',
       'Status Change Date', 'Subregion Grouping', 'Subscription End Date',
       'Subscription Start Date', 'Subscription Start Year', 'WIP Flag'],
      dtype='object')

In [50]:
# customer ids cols
ECH_ecrid_col = 'ecrid'
journals_ecrid_col = 'SIS Id  (Agreement SIS)'
other_ecrid_col = 'SIS Id  (Agreement SIS)'
churn_activities_ecrid_col = 'ECR Id'
churn_risks_ecrid_col = 'Account Name: ECR Id'
account_assignment_ecrid_col = 'ECRID'
NPS_ecrid_col = 'ECR_ID'
usage_ecr_id = 'ECR_ID'
interactions_ecrid_col = 'ECR_ID'
cancellations_ecrid_col = 'SIS Id  (Agreement SIS)'

In [23]:
# customer ids cols
ECH_ecrid_col = 'ecrid'
journals_ecrid_col = 'HQ SIS Id (Agreement SIS)'
other_ecrid_col = 'HQ SIS Id (Agreement SIS)'
churn_activities_ecrid_col = 'ECR Id'
churn_risks_ecrid_col = 'Account Name: ECR Id'
account_assignment_ecrid_col = 'ECRID'
NPS_ecrid_col = 'ECR_ID'
usage_ecr_id = 'ECR_ID'
interactions_ecrid_col = 'ECR_ID'
cancellations_ecrid_col = 'HQ SIS Id (Agreement SIS)'

In [51]:
# Customers and contracts

ECH_customers = ECH[ECH_ecrid_col].unique()

journals_customers = journals_contracts[journals_ecrid_col].unique()
other_customers = other_contracts[other_ecrid_col].unique()
churn_activity_customers = churn_activities[churn_activities_ecrid_col].unique().astype(str)
churn_risks_customers = churn_risks[churn_risks_ecrid_col].unique().astype(str)
account_assignment_customers = account_assignment[account_assignment_ecrid_col].unique().astype(str)
NPS_customers = NPS[NPS_ecrid_col].unique().astype(str)
usage_customers = usage[usage_ecr_id].unique().astype(str)
interactions_customers = interactions[interactions_ecrid_col].unique().astype(str)
cancellation_customers = cancellations[cancellations_ecrid_col].unique().astype(str)

#venn2([set(['A', 'B', 'C', 'D']), set(['D', 'E', 'F'])])
#plt.show()


In [52]:
print(f'Unique customers in ECH {len(ECH_customers)}')
print(f'Unique customers in journals {len(journals_customers)}')
print(f'Unique customers in other_contracts {len(other_customers)}')
print(f'Unique customers in churn_activity_customers {len(churn_activity_customers)}')
print(f'Unique customers in churn_risks_customers {len(churn_risks_customers)}')
print(f'Unique customers in account_assignment_customers {len(account_assignment_customers)}')
print(f'Unique customers in NPS_customers {len(NPS_customers)}')
print(f'Unique customers in usage_customers {len(usage_customers)}')
print(f'Unique customers in interactions_customers {len(interactions_customers)}')
print(f'Unique customers in cancellations {len(cancellation_customers)}')



Unique customers in ECH 137546
Unique customers in journals 14033
Unique customers in other_contracts 9325
Unique customers in churn_activity_customers 25903
Unique customers in churn_risks_customers 40009
Unique customers in account_assignment_customers 68078
Unique customers in NPS_customers 8464
Unique customers in usage_customers 22098
Unique customers in interactions_customers 25444
Unique customers in cancellations 8586


### 1.3 million ECR-IDs in whole data. is it just customers in Sales Force

### Mapping IDs is best appoarch
### makeshift approach is to append ECR


### Account assignment 
cascades down not upwards
so if parent is assigned then all children area also assigned

Hierarchy Data:
Parent Child
A      X
A      Y
Y      Z
X      Y
X      Z
X      X

designed to match Parent column to available ECR-ID
column: Hierarcy Level - 1 for child 2 for grand child


In [53]:
# Common customers between ECH and journals

len(np.intersect1d(ECH_customers, journals_customers, assume_unique=True))

14021

In [27]:
# Common customers between ECH and other

len(np.intersect1d(ECH_customers, other_customers, assume_unique=True))

7668

In [28]:
# Common customers between ECH and Cancellations

len(np.intersect1d(ECH_customers, cancellation_customers, assume_unique=True))

7730

In [29]:
# Common customers between ECH and Churn activity

len(np.intersect1d(ECH_customers, churn_activity_customers, assume_unique=True))

25875

In [30]:
# Common customers between ECH and Churn risk customers

len(np.intersect1d(ECH_customers, churn_risks_customers, assume_unique=True))

39870

In [31]:
# Common customers between ECH and account_assignment_customers

len(np.intersect1d(ECH_customers, account_assignment_customers, assume_unique=True))

68030

In [32]:
# Common customers between ECH and usage_customers

len(np.intersect1d(ECH_customers, usage_customers, assume_unique=True))

22063

In [33]:
# Common customers between ECH and interactions_customers

len(np.intersect1d(ECH_customers, interactions_customers, assume_unique=True))

17798

In [34]:
# Common customers between ECH and NPS_customers

len(np.intersect1d(ECH_customers, NPS_customers, assume_unique=True))

7764

# Merge 

In [35]:
cust_journals = pd.merge(ECH, journals_contracts, how='inner', left_on=ECH_ecrid_col, right_on=journals_ecrid_col)

cust_others = pd.merge(ECH, other_contracts, how='inner', left_on=ECH_ecrid_col, right_on=other_ecrid_col)

cust_cancelled = pd.merge(ECH, cancellations, how='inner', left_on=ECH_ecrid_col, right_on=cancellations_ecrid_col)

Index(['ecrid', 'name', 'city', 'Country ISO', 'Region', 'consortium',
       'post_code', 'Classification'],
      dtype='object')

In [34]:
journals_contracts.columns

Index(['Agreement End Date', 'Agreement Number', 'Agreement Start Date',
       'Bookigns - Committed Print(Rep)',
       'Bookings - Final Net Price - Agent Discount Amount(Rep)',
       'Business Division (Agreement SIS)', 'Business Indicator',
       'Calculated New/Renewal', 'Country Name (Agreement SIS)', 'Division',
       'HQ SIS Id (Agreement SIS)', 'Invoice Date', 'Invoice Num',
       'Name  (Agreement SIS)', 'Parent Agreement Number', 'Payment Term',
       'Payment Term Description', 'Payment Term Type', 'Product Line Level 1',
       'Product Line Level 2', 'Product Line Level 3', 'Product Line Level 4',
       'Product Revenue Type', 'RSO', 'Renewal Exp Complete Date',
       'SIS Id  (Agreement SIS)', 'Saleable Product Name (Source)',
       'Sales Division (Agreement SIS)', 'Sales Type', 'Status',
       'Status Change Date', 'Subregion Grouping', 'Subscription End Date',
       'Subscription Start Date', 'Subscription Start Year', 'WIP Flag'],
      dtype='object')

## Journals

### Journals revenue by customers

In [36]:
pd.options.display.float_format = '${:,.0f}'.format


cust_journals.groupby(
    ['ecrid', 'name','Country ISO']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

## also good idea to look at the the consortium number. same as consortium account.
## sis_id or consortia id

## negative values look like credit agreements
## customer doest want to sign agreement, but already charged 

,,,revenue,num_agreements
ecrid,name,Country ISO,,
ECR-490216,University Consortium for Digital Journals,FR,"$246,085,921",18
ECR-310775,Coordination of Higher Education Personnel Improvement,BR,"$207,604,102",118
ECR-71655,Ohio Library and Information Network,US,"$86,269,246",21
ECR-380603,Collaboration of the Dutch university libraries and the Koninklijke Bibliotheek,NL,"$72,974,384",31
ECR-23550,US Department of Energy,US,"$60,426,496",134
...,...,...,...,...
ECR-1136006,Australia and New Zealand Hepatic Pancreatic and Biliary Association Inc,AU,$-0,1
ECR-384319,Eisenhower Medical Center,US,$-0,1
ECR-789986,Canterbury District Health Board,NZ,$-0,1


### Journals Spend by country

In [37]:
cust_journals.groupby(
    ['Country ISO']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

#just ensure Japan revenue is correct

,revenue,num_agreements
Country ISO,,
US,"$2,809,446,769",14718
JP,"$630,781,220",8657
CN,"$536,126,776",3394
GB,"$417,148,205",3110
FR,"$322,596,271",2620
...,...,...
AG,"$2,081",1
SV,"$1,799",1
YT,"$1,568",1


### Journals Revenue by product 

In [38]:
cust_journals.groupby(
    ['Product Line Level 2']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

# check later

,revenue,num_agreements
Product Line Level 2,,
Journals Electronic Subscriptions,"$8,428,964,698",49100
Journal Level Sales,"$394,939,231",29656
Journals Transactions,"$106,998,010",6983
Journals Backfiles,"$91,294,349",2473
Journals Legacy,"$2,418,336",31
Journals Other,"$226,755",8


In [39]:
cust_journals.groupby(
    ['ecrid', 'name','Product Line Level 2']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False)

,,,revenue,num_agreements
ecrid,name,Product Line Level 2,,
ECR-490216,University Consortium for Digital Journals,Journals Electronic Subscriptions,"$245,815,686",18
ECR-310775,Coordination of Higher Education Personnel Improvement,Journals Electronic Subscriptions,"$195,405,547",93
ECR-71655,Ohio Library and Information Network,Journals Electronic Subscriptions,"$82,842,911",20
ECR-380603,Collaboration of the Dutch university libraries and the Koninklijke Bibliotheek,Journals Electronic Subscriptions,"$71,366,227",21
ECR-23550,US Department of Energy,Journals Electronic Subscriptions,"$59,092,278",106
...,...,...,...,...
ECR-370079,Helen Hayes Hospital,Journals Electronic Subscriptions,$-0,1
ECR-375279,HeBIS Consortium,Journal Level Sales,$-0,3
ECR-334114,Royal Agricultural University,Journals Electronic Subscriptions,$-193,8


### Customer journals revenue by year

In [40]:
cust_journals_year = cust_journals.groupby(
    ['ecrid', 'name','Subscription Start Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(cust_journals_year, 
               index=['ecrid','name'], 
               columns=['Subscription Start Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

revenue                                                                                          
Subscription Start Year                                                  2015           2016           2017           2018           2019           2020            All
ecrid       name                                                                                                                                                       
All                                                            $1,421,433,642 $1,493,287,164 $1,525,262,135 $1,557,748,436 $1,519,143,953 $1,507,966,049 $9,024,841,379
ECR-490216  University Consortium for Digital Journals            $39,256,562    $40,606,841    $41,735,252    $42,993,876    $41,586,168    $39,907,222   $246,085,921
ECR-310775  Coordination of Higher Education Personnel Impr...    $26,942,344    $25,549,101    $60,972,810    $30,129,666    $30,184,230    $33,825,950   $207,604,102
ECR-71655   Ohio Library and Information Network                  $13,152,575    $14,102,916    $14,041,722    $14,483,386    $15,086,284    $15,402,363    $86,269,246
ECR-380603  Collaboration of the Dutch university libraries...             $0    $14,182,679    $14,489,812    $14,794,098    $14,794,098    $14,713,696    $72,974,384
...                                                                       ...            ...            ...            ...            ...            ...            ...
ECR-1136006 Australia and New Zealand Hepatic Pancreatic an...             $0            $-0             $0             $0             $0             $0            $-0
ECR-789986  Canterbury District Health Board                              $-0             $0             $0             $0             $0             $0            $-0
ECR-384319  Eisenhower Medical Center                                     $-0             $0             $0             $0             $0             $0            $-0
ECR-596603  Italian Society of Gynecology and Obstetrics                  $-0             $0             $0             $0             $0             $0            $-0
ECR-1413    University of Essex                                       $36,369        $39,940       $-30,774       $-45,846       $-64,542       $-76,242      $-141,096

[12667 rows x 7 columns]

In [42]:

product_journals_year = cust_journals.groupby(
    ['Product Line Level 2','Subscription Start Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(product_journals_year, 
               index=['Product Line Level 2'], 
               columns=['Subscription Start Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)



revenue                                                                                          
Subscription Start Year                     2015           2016           2017           2018           2019           2020            All
Product Line Level 2                                                                                                                      
All                               $1,421,433,642 $1,493,287,164 $1,525,262,135 $1,557,748,436 $1,519,143,953 $1,507,966,049 $9,024,841,379
Journals Electronic Subscriptions $1,325,832,291 $1,391,119,910 $1,421,020,576 $1,446,056,033 $1,413,072,591 $1,431,863,297 $8,428,964,698
Journal Level Sales                  $53,673,243    $62,055,950    $67,553,119    $70,719,164    $69,411,038    $71,526,716   $394,939,231
Journals Transactions                $19,076,951    $21,049,240    $19,255,780    $21,978,348    $24,473,558     $1,164,132   $106,998,010
Journals Backfiles                   $20,426,446    $19,010,023    $17,368,806    $18,987,836    $12,139,498     $3,361,739    $91,294,349
Journals Legacy                       $2,418,336             $0             $0             $0             $0             $0     $2,418,336
Journals Other                            $6,374        $52,041        $63,853         $7,054        $47,267        $50,165       $226,755

## Other products

### Other products Revenue by customer

In [43]:
cust_others.groupby(
    ['ecrid', 'name','Country ISO']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

,,,revenue,num_agreements
ecrid,name,Country ISO,,
ECR-310775,Coordination of Higher Education Personnel Improvement,BR,"$50,928,118",83
ECR-46095,Russian National Public Library for Science and Technology,RU,"$24,880,869",16
ECR-507508,Sanofi SA,FR,"$24,480,383",172
ECR-373436,Consortium Polish Academics,PL,"$22,124,301",28
ECR-459818,Ministry of Science Technology and Productive Innovation,AR,"$22,120,632",40
...,...,...,...,...
ECR-927783,Erzurum Technical University,TR,$0,5
ECR-244125,National University of Asuncion,PY,$0,2
ECR-32232652,Ada Kent University,TR,$0,2


### Revenue by product

In [44]:
cust_others.groupby(
    ['Product Line Level 2']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

# print might not have agreement
# exclude HS and CS
# Stick to RSS

,revenue,num_agreements
Product Line Level 2,,
SCOPUS,"$627,004,895",11410
Reaxys,"$323,221,047",5127
eBooks,"$256,484,558",28687
EV,"$207,353,675",5537
embase,"$188,457,542",2967
...,...,...
Mendeley Institutional Edition,"$3,836,270",2873
Entellect,"$505,442",11
Research Data Management,"$504,454",12


### customer revenue by year

In [45]:
cust_others_year = cust_others.groupby(
    ['ecrid', 'name','Subscription Start Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(cust_others_year, 
               index=['ecrid','name'], 
               columns=['Subscription Start Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

revenue                                                                                
Subscription Start Year                                               2015         2016         2017         2018         2019         2020            All
ecrid      name                                                                                                                                           
All                                                           $353,165,309 $373,097,090 $388,979,669 $405,519,545 $416,721,622 $378,019,735 $2,315,502,969
ECR-310775 Coordination of Higher Education Personnel Impr...  $14,881,447   $5,450,537   $8,024,876   $6,995,625   $7,022,095   $8,553,540    $50,928,118
ECR-46095  Russian National Public Library for Science and...   $2,150,847   $2,343,348   $2,130,855   $5,276,635   $6,160,358   $6,818,825    $24,880,869
ECR-507508 Sanofi SA                                            $3,929,178   $4,242,481   $4,076,278   $4,266,318   $4,019,883   $3,946,244    $24,480,383
ECR-373436 Consortium Polish Academics                          $4,972,453   $2,572,798   $2,604,844   $3,402,227   $4,231,780   $4,340,200    $22,124,301
...                                                                    ...          ...          ...          ...          ...          ...            ...
ECR-545376 Turkish National Police Academy                              $0           $0           $0           $0           $0           $0             $0
ECR-379238 National Institute of Oceanography and Fisheries             $0           $0           $0           $0           $0           $0             $0
ECR-629768 Autonomous University of Chiapas                             $0           $0           $0           $0           $0           $0             $0
ECR-934572 Technological Institute of Ciudad Victoria                   $0           $0           $0           $0           $0           $0             $0
ECR-709815 Siberian Chemistry Institutes                               $-0           $0           $0           $0           $0           $0            $-0

[7669 rows x 7 columns]

### product revenue by year

In [46]:
product_others_year = cust_others.groupby(
    ['Product Line Level 2','Subscription Start Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(product_others_year, 
               index=['Product Line Level 2'], 
               columns=['Subscription Start Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)



revenue                                                                                
Subscription Start Year                2015         2016         2017         2018         2019         2020            All
Product Line Level 2                                                                                                       
All                            $353,165,309 $373,097,090 $388,979,669 $405,519,545 $416,721,622 $378,019,735 $2,315,502,969
SCOPUS                          $93,842,395  $92,649,408  $99,822,742 $109,052,833 $114,411,226 $117,226,291   $627,004,895
Reaxys                          $50,089,900  $52,620,483  $54,766,577  $54,769,866  $56,270,010  $54,704,212   $323,221,047
eBooks                          $46,055,561  $50,229,708  $49,776,893  $46,492,241  $44,659,444  $19,270,711   $256,484,558
EV                              $34,198,534  $35,297,624  $34,378,453  $33,684,218  $34,988,712  $34,806,134   $207,353,675
...                                     ...          ...          ...          ...          ...          ...            ...
Mendeley Institutional Edition     $517,186     $631,080     $734,639     $755,729     $657,620     $540,017     $3,836,270
Entellect                                $0           $0           $0     $125,000     $130,442     $250,000       $505,442
Research Data Management                 $0           $0           $0     $130,500     $287,762      $86,192       $504,454
Copyrights                               $0           $0           $0      $29,084      $81,332      $59,668       $170,083
ScienceMetrix                            $0           $0           $0           $0      $46,900      $41,275        $88,175

[27 rows x 7 columns]

## Cancelled contracts

### Cancelled revenue by customer

In [47]:
cust_cancelled.groupby(
    ['ecrid', 'name', 'Country ISO']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

,,,revenue,num_agreements
ecrid,name,Country ISO,,
ECR-1004,University of California System,US,"$10,883,968",24
ECR-375272,Konsortium Baden-Wurttemberg,DE,"$6,013,895",1
ECR-44401,Hungarian Academy of Sciences,HU,"$5,197,226",3
ECR-928880,Centre for Research On Scientific and Technical Information,DZ,"$4,856,530",4
ECR-339079,Ministry of Higher Education Scientific Research and Technology and Information and Communication Technologies,TN,"$3,949,677",1
...,...,...,...,...
ECR-365981,Health and Safety Authority,IE,$-0,1
ECR-417308,Forest University,CO,$-0,2
ECR-737806,University Colleges of Columbia,CO,$-0,1


### Cancelled revenue by product

In [48]:
cust_cancelled.groupby(
    ['Product Line Level 2']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

,revenue,num_agreements
Product Line Level 2,,
Journals Electronic Subscriptions,"$173,465,915",3810
CK Physician,"$25,987,050",646
SCOPUS,"$23,727,730",842
Reaxys,"$17,066,727",451
Journal Level Sales,"$12,301,681",2311
...,...,...
SSRN,"$331,915",106
Life Sciences Other,"$287,551",8
CK Med Ed,"$137,429",8


### Cancellations by customer and year

In [49]:
cust_cancelled_year = cust_cancelled.groupby(
    ['ecrid', 'name', 'Cancellation Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(cust_cancelled_year, 
               index=['ecrid','name'], 
               columns=['Cancellation Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

KeyError: 'Cancellation Year'

### Product cancellations by year

In [90]:
product_cancelled_year = cust_cancelled.groupby(
    ['Product Line Level 2','Cancellation Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(product_cancelled_year, 
               index=['Product Line Level 2'], 
               columns=['Cancellation Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

revenue                                                                                  
Cancellation Year                        2015        2016        2017        2018        2019        2020     2021          All
Product Line Level 2                                                                                                           
All                               $36,958,283 $42,878,972 $69,302,257 $95,173,482 $70,363,467 $14,975,093 $142,444 $329,793,998
Journals Electronic Subscriptions $14,304,948 $15,702,864 $40,462,644 $59,094,604 $37,172,007  $6,619,770 $109,078 $173,465,915
CK Physician                       $3,515,455  $4,757,864  $5,960,471  $5,825,478  $5,822,804    $104,979       $0  $25,987,050
SCOPUS                             $4,532,807  $5,148,291  $3,672,445  $4,487,878  $4,253,345  $1,632,964       $0  $23,727,730
Reaxys                             $2,921,846  $2,189,584  $2,889,784  $3,584,287  $3,627,993  $1,853,233       $0  $17,066,727
...                                       ...         ...         ...         ...         ...         ...      ...          ...
SSRN                                       $0          $0      $4,799    $184,973    $140,291      $2,652       $0     $332,715
Life Sciences Other                   $33,684     $84,228    $102,737          $0     $66,903          $0       $0     $287,551
CK Med Ed                                  $0          $0          $0      $6,399    $131,031          $0       $0     $137,429
Journals Legacy                            $0    $103,227          $0          $0          $0          $0       $0     $103,227
Journals Other                             $0          $0     $45,885          $0          $0          $0       $0      $45,885

[35 rows x 8 columns]

In [91]:
product_cancelled_year = cust_cancelled.groupby(
    ['Product Line Level 3','Cancellation Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(product_cancelled_year, 
               index=['Product Line Level 3'], 
               columns=['Cancellation Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

revenue                                                                                  
Cancellation Year                      2015        2016        2017        2018        2019        2020     2021          All
Product Line Level 3                                                                                                         
All                             $36,958,283 $42,878,972 $69,302,257 $95,173,482 $70,363,467 $14,975,093 $142,444 $329,793,998
E-fees                           $5,956,968  $5,220,927 $23,147,395 $28,576,178 $20,727,686  $2,797,317   $3,137  $86,429,607
Journal Collections              $7,606,413  $9,321,215 $16,122,742 $28,170,217 $15,639,589  $3,368,430 $105,736  $80,334,341
Scopus                           $4,532,807  $5,148,291  $3,657,445  $4,412,878  $4,247,313  $1,614,573       $0  $23,613,307
ClinicalKey Global               $3,325,261  $4,402,658  $4,931,628  $5,265,285  $4,700,726     $51,352       $0  $22,676,910
...                                     ...         ...         ...         ...         ...         ...      ...          ...
Clinical Pharmacology RD 1               $0          $0          $0          $0          $0          $0       $0           $0
MDC Specialty Edition                    $0          $0          $0          $0          $0          $0       $0           $0
MDC ANZ Standard Subscr                  $0          $0          $0          $0          $0          $0       $0           $0
MD Consult Link                          $0          $0          $0          $0          $0          $0       $0           $0
Natural Medicines Database Link          $0          $0          $0          $0          $0          $0       $0           $0

[160 rows x 8 columns]

## Hierarchy Mapping

In [55]:
# Mao Parent and Children